<a href="https://colab.research.google.com/github/smriti-19/hello-world/blob/main/Spoken_Digit_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import os
from sklearn.preprocessing import normalize
import random
from scipy.fftpack import fft
from scipy.io import wavfile # get the api

In [ ]:
# calculating the modulus of the signal
def mod(v1):
    sum=0
    for i in range(v1.shape[0]):
        sum+=v1[i]**2
    return sum/2

#determining the correlation between two signals
def correlation(v1,v2):
    corr = 0
    for i in range(v1.shape[0]):
        corr+=v1[i]*v2[i]
    return corr



In [ ]:
# creating the reference samples by averaging out the frequency spectrum for each digit
fft_list = list()
for j in range(25,50):
    each_list = list()
    for i in range(10):

        # read the .wav file
        fs, data = wavfile.read('/content/drive/MyDrive/free-spoken-digit-dataset/recordings/'+ str(i)+'_jackson_'+str(j)+'.wav') # load the data

        a = list(data)  # this is a two channel soundtrack, I get the first track

        b=[(ele/2**8.)*2-1 for ele in a] # this is 8-bit track, b is now normalized on [-1,1)

        c = fft(b) # calculate fourier transform (complex numbers list)

        d = int(len(c)/2)  # you only need half of the fft list (real signal symmetry)

        # pad the sequence so that all spectrums are of the same length (5000)
        if(d-1>5000):
            each_list.append(abs(c[:5000]))
            print(abs(c[:5000].shape))
        else:
            q = np.pad(abs(c[:d-1]),(0,5000-(d-1)),'constant')
        q = np.reshape(q,(q.shape[0],1))
        q = normalize(q, axis = 0)
        q = np.reshape(q,(q.shape[0]))
        each_list.append(q)

    fft_list.append(each_list)
data = np.array(fft_list)

# average taken for all samples
final_data = np.average(data,axis=0)
fft_list = list(final_data)


# depicting the performance of the idea using autocorrelation from reference samples
for j in range(10):
    corr_list=list()
    for i in range(10):
        corr_list.append(correlation(fft_list[j],fft_list[i])-mod(fft_list[i]))
    # ax = plt.subplot(10,1,j+1)
    plt.plot(corr_list)
    plt.title(str(j)+ " correlation")
    plt.show()



In [ ]:
directory ="drive/My Drive/free-spoken-digit-dataset/recordings/"
num=0

# testing the work using data samples for the test dataset
for k in range(10):
    for l in range(0,25): # change this for test set
        random_file = str(k)+'_jackson_' + str(l)+'.wav'
        print(random_file)
        fs_test, data_test = wavfile.read(directory+random_file) # load the data
        a_test = data_test
        b_test=[(ele/2**8.)*2-1 for ele in a_test]
        c_test = fft(b_test) # calculate fourier transform (complex numbers list)

        d = int(len(c_test)/2)

        # padding sequences

        if(d-1>5000):
            fft_list.append(abs(c_test[:5000]))
            q = abs(c_test[:5000].shape)
        else:
            q = np.pad(abs(c_test[:d-1]),(0,5000-(d-1)),'constant')
        q = np.reshape(q,(q.shape[0],1))
        q = normalize(q, axis = 0)
        q = np.reshape(q,(q.shape[0]))
        corr_list_test = list()
        max_num = correlation(q,fft_list[0])-mod(fft_list[0])
        max_index = 0
        for i in range(10):

            # determining the maximum achieved after cross-correlation

            corr_list_test.append(correlation(q,fft_list[i])-mod(fft_list[i]))
            if(max_num<correlation(q,fft_list[i])-mod(fft_list[i])):
                max_num = correlation(q,fft_list[i])-mod(fft_list[i])
                max_index = i
        if(max_index==k):
            num+=1  # correctly classified examples

print(str(num/5) + "% Is the accuracy of the system")
print("Predicted number is " + str(max_index))
plt.plot(corr_list_test)
plt.show()


In [ ]:
def read_wave_file(filename, template_size):
    # read the .wav file
    fs, data = wavfile.read(filename) # load the data

    soundtrack_signal = list(data)  # this is a two channel soundtrack, I get the first track

    time_signal =[(ele/2**8.)*2-1 for ele in soundtrack_signal] # this is 8-bit track, b is now normalized on [-1,1)

    freq_signal = fft(time_signal) # calculate fourier transform (complex numbers list)

    d = len(freq_signal)//2  # you only need half of the fft list (real signal symmetry)

    # pad the sequence so that all spectrums are of the same length (template_size)
    fft_signal = normalize_signal_length(freq_signal, template_size, d)
    

    time_signal = np.array(time_signal)
    time_signal = normalize_signal_length(time_signal, template_size, len(time_signal))
    

    return fft_signal, time_signal


def normalize_signal_length(signal, template_size, intended_size):

    if(intended_size-1>template_size):
        signal = abs(signal[:template_size])
        
    else:
        signal = np.pad(abs(signal[:intended_size-1]),(0,template_size-(intended_size-1)),'constant')
    signal = np.reshape(signal,(signal.shape[0],1))
    signal = normalize(signal, axis = 0)
    signal = np.reshape(signal,(signal.shape[0]))
    
    return signal

def predict_digit(digit_template, signal):
    max_correlation = 0
    number_identified = -1
    for k in range(len(digit_template)):
        corr = correlation(signal, digit_template[k]) - mod(digit_template[k])
        if corr>max_correlation:
          max_correlation = corr
          number_identified = k
    return number_identified, max_correlation
        



In [ ]:
from tempfile import template
from numpy.lib.function_base import average
fft_digit_template = list()
time_digit_template = list()
template_size = 2000
directory = '/content/drive/MyDrive/free-spoken-digit-dataset/recordings/'
for i in range(10):
    fft_signal_template = np.zeros((template_size))
    time_signal_template = np.zeros((template_size))
    num_examples = 0
    for j in range(15,50):

        # read the .wav file

        audio_file = directory + str(i)+'_jackson_'+str(j)+'.wav'
        fft_signal, time_signal  = read_wave_file(audio_file, template_size)
        fft_signal_template +=fft_signal
        time_signal_template += time_signal
        num_examples+=1

    fft_digit_template.append(fft_signal_template/num_examples)
    time_digit_template.append(time_signal_template/num_examples)


In [ ]:
true_labels = list()
pred_labels = list()
count = 0
for i in range(10):
    
    for j in range(0,15):

        # read the .wav file
        filename = directory + str(i)+'_jackson_'+str(j)+'.wav'
        fft_signal, time_signal = read_wave_file(filename, template_size)
        num_time, max_cor_time = predict_digit(time_digit_template, time_signal)
        num_freq, max_cor_freq = predict_digit(fft_digit_template, fft_signal)

        
        true_labels.append(i)
        
        pred_labels.append(num_freq)

        

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(true_labels, pred_labels)

0.5866666666666667

In [ ]:
print(count)

28


0.18666666666666668

In [ ]:
"""
3 experiments
1. Average signals in time domain from train set and use that as template for cross-correlation matching -> score of 41.33%
2. Average signals in freq domain from train set and use that as template for cross-correlation matching -> score of 58.67%
3. Taking maximum value of correlation from both time and frequency for matching -> score of 46%




"""